In [9]:
import time
import json
import sys
import os

from globus_automate_client import (create_flows_client, graphviz_format, state_colors_for_log,
                                    create_action_client, 
                                    create_flows_client)
from funcx.sdk.client import FuncXClient
from funcx.serialize import FuncXSerializer

In [10]:
fxc = FuncXClient()
# fxs = FuncXSerializer() # (de)serialize the funcx codes

In [11]:
def copy_file(params):
    import os
    import shutil
    
    ifpath = params['ifpath']
    ofpath = params['ofpath']
    
    #shutil.copyfile(ifpath, ofpath)
    shutil.copyfile(ifpath, ofpath)

def my_hello():
    return "Welcome to the machine"

func_cp_uuid = fxc.register_function(copy_file)
func_hl_uuid = fxc.register_function(my_hello)
print(func_cp_uuid)

74a40c9e-419b-4b60-8681-e818df88cc15


In [3]:
def ptycho(data):
    """Test the ptycho tool"""
    import os
    import subprocess
    from subprocess import PIPE
    
    python_path = data['python_path']
    script_path = data['script_path']
    
    #recon. script parameters
    ifpath = data['ifpath']
    ofpath = data['ofpath']
    rec_alg = data['rec_alg']
    rec_nmodes = data['rec_nmodes']
    rec_niter = data['rec_niter']
    rec_output_freq = data['rec_output_freq']
    rec_recover_psi = '--recover-psi' if (('rec_recover_psi' in data) and data['rec_recover_psi']) else ''
    rec_recover_probe = '--recover-probe' if (('rec_recover_probe' in data) and data['rec_recover_probe']) else ''
    rec_recover_positions = '--recover-positions' if (('rec_recover_positions' in data) and data['rec_recover_positions']) else ''
    rec_model = data['rec_model']
    rec_ngpu = data['rec_ngpu']
    rec_use_mpi = '--use-mpi' if (('rec_use_mpi' in data) and data['rec_use_mpi']) else ''
    
    try:
        os.mkdir(ofpath)
    except:
        pass
    
    cmd = f"{python_path} {script_path} --algorithm={rec_alg} --nmodes={rec_nmodes} --niter={rec_niter} --output-freq={rec_output_freq} {rec_recover_psi} {rec_recover_probe} {rec_recover_positions} --model={rec_model} --ngpu={rec_ngpu} {rec_use_mpi} --ifile='{ifpath}' --folder='{ofpath}'"

    try:
        res = subprocess.run(cmd, stdout=PIPE, stderr=PIPE,
                             shell=True, executable='/bin/bash')
    except:
        pass
    outstr = f"{res.stdout}"
    return outstr
    
func_ptycho_uuid = fxc.register_function(ptycho)
print(func_ptycho_uuid)

ba8c653b-713e-43ac-85f9-d615b72c6b8a


In [4]:
CLIENT_ID = "e6c75d97-532a-4c88-b031-8584a319fa3e"

In [5]:
flow_definition = {
  "Comment": "An analysis flow",
  "StartAt": "Transfer",
  "States": {
    "Transfer": {
      "Comment": "Initial transfer",
      "Type": "Action",
      "ActionUrl": "https://actions.automate.globus.org/transfer/transfer",
      "ActionScope": "https://auth.globus.org/scopes/actions.globus.org/transfer/transfer",
      "Parameters": {
        "source_endpoint_id.$": "$.input.source_endpoint", 
        "destination_endpoint_id.$": "$.input.dest_endpoint",
        "transfer_items": [
          {
            "source_path.$": "$.input.source_path",
            "destination_path.$": "$.input.dest_path",
            "recursive": False
          }
        ]
      },
      "ResultPath": "$.Transfer1Result",
      "WaitTime": 600,
      "Next": "Analyze"
    },
    "Analyze": {
      "Comment": "Run a funcX function",
      "Type": "Action",
      "ActionUrl": "https://api.funcx.org/automate",
      "ActionScope": "https://auth.globus.org/scopes/facd7ccc-c5f4-42aa-916b-a0e270e2c2a9/automate2",
      "Parameters": {
          "tasks": [{
            "endpoint.$": "$.input.fx_ep",
            "func.$": "$.input.fx_id",
            "payload.$": "$.input.params"
        }]
      },
      "ResultPath": "$.AnalyzeResult",
      "WaitTime": 1800,
      "Next": "Transfer2"
    },
    "Transfer2": {
      "Comment": "Return transfer",
      "Type": "Action",
      "ActionUrl": "https://actions.automate.globus.org/transfer/transfer",
      "ActionScope": "https://auth.globus.org/scopes/actions.globus.org/transfer/transfer",
      "Parameters": {
        "source_endpoint_id.$": "$.input.dest_endpoint", 
        "destination_endpoint_id.$": "$.input.source_endpoint",
        "transfer_items": [
          {
            "source_path.$": "$.input.result_path",
            "destination_path.$": "$.input.source_result_path",
            "recursive": True #False
          }
        ]
      },
      "ResultPath": "$.Transfer2Result",
      "WaitTime": 600,
      "End": True
    },
  }
}

In [6]:
flows_client = create_flows_client(CLIENT_ID)
flow = flows_client.deploy_flow(flow_definition, title="Simple ptycho data analysis flow")
flow_id = flow['id']

print(flow)

flow_scope = flow['globus_auth_scope']

#print(f'Newly created flow with id:\n{flow_id}\nand scope:\n{flow_scope}')

GlobusHTTPResponse({'action_url': 'https://flows.globus.org/flows/0414640d-933a-4978-bb98-817cba475b55', 'administered_by': [], 'api_version': '1.0', 'created_by': 'urn:globus:auth:identity:a478e9c0-d274-11e5-9901-63aaec352b80', 'definition': {'Comment': 'An analysis flow', 'StartAt': 'Transfer', 'States': {'Analyze': {'ActionScope': 'https://auth.globus.org/scopes/facd7ccc-c5f4-42aa-916b-a0e270e2c2a9/automate2', 'ActionUrl': 'https://api.funcx.org/automate', 'Comment': 'Run a funcX function', 'Next': 'Transfer2', 'Parameters': {'tasks': [{'endpoint.$': '$.input.fx_ep', 'func.$': '$.input.fx_id', 'payload.$': '$.input.params'}]}, 'ResultPath': '$.AnalyzeResult', 'Type': 'Action', 'WaitTime': 1800}, 'Transfer': {'ActionScope': 'https://auth.globus.org/scopes/actions.globus.org/transfer/transfer', 'ActionUrl': 'https://actions.automate.globus.org/transfer/transfer', 'Comment': 'Initial transfer', 'Next': 'Analyze', 'Parameters': {'destination_endpoint_id.$': '$.input.dest_endpoint', 'sou

In [7]:
theta_fx_endpoint = 'f765db7a-038c-47ea-9176-d81de31c054f' #FuncX endpoint

#src_endpoint = '9c9cb97e-de86-11e6-9d15-22000a1e3b52' #'aps#data' # Voyager
src_endpoint = 'dd916908-0072-11e7-badc-22000b9a448b' #'hostel' #aps/workstation
dest_endpoint = '08925f04-569f-11e7-bef8-22000b9a448b' #'alcf#dtn_theta' # Theta DTN

#src_filepath = '/gdata/RAVEN/bicer/2020-1/comm_33IDD/extracted_tekin'
src_filepath = '/prisma-data1/bicer/workflow/input'
src_filename = 'extracted_scan300.h5'
#src_result_path = '/gdata/RAVEN/bicer/2020-1/comm_33IDD/globus_automate_results'
src_result_path = '/prisma-data1/bicer/workflow/result'

dest_filepath = '/lus/grand/projects/hp-ptycho/bicer/ptycho/comm_33IDD/globus_automate/transferred'
dest_resultpath = '/lus/grand/projects/hp-ptycho/bicer/ptycho/comm_33IDD/globus_automate/result'


# Ptycho recon params
script_path = '/home/bicer/projects/tike/scripts/mpi_tike-recon.py'
python_path = "/home/bicer/miniconda3/envs/ptycho/bin/python"

rec_alg = 'cgrad'
rec_nmodes = 8
rec_upd_pos = False
rec_niter = 30
rec_output_freq = 10
rec_recover_psi = True
rec_recover_probe= True
rec_recover_positions = False
rec_model = 'gaussian'
rec_ngpu = 8
rec_use_mpi = True

flow_input = {
    "input": {
        "source_endpoint": f"{src_endpoint}",
        "source_path": f"{src_filepath}/{src_filename}",
        "dest_endpoint": dest_endpoint,
        "dest_path": f"{dest_filepath}/{src_filename}",
        
        "result_path": f"{dest_resultpath}",#f"{dest_resultpath}/out_{src_filename}",
        "source_result_path": f"{src_result_path}", #/out_{src_filename}",
        "fx_ep": f"{theta_fx_endpoint}",
        #"fx_id": f"{func_cp_uuid}", #f"{func_hl_uuid}" #
        "fx_id": f"{func_ptycho_uuid}",
        "params": {'ifpath': f"{dest_filepath}/{src_filename}",
                   'ofpath': f"{dest_resultpath}/",
                   'script_path': script_path,
                   'python_path': python_path,
                   'rec_alg': rec_alg,
                   'rec_nmodes': rec_nmodes,
                   'rec_upd_pos': rec_upd_pos,
                   'rec_niter': rec_niter,
                   'rec_output_freq': rec_output_freq,
                   'rec_recover_psi': rec_recover_psi,
                   'rec_recover_probe': rec_recover_probe,
                   'rec_recover_positions': rec_recover_positions,
                   'rec_model': rec_model,
                   'rec_ngpu': rec_ngpu,
                   'rec_use_mpi': rec_use_mpi}
        
        #"params": {"ifpath": f"{dest_filepath}/{src_filename}",
        #            "ofpath": f"{dest_resultpath}/out_{src_filename}"}
    }
}

print(f"transfer file from {src_endpoint}#{src_filepath}/{src_filename} to {dest_endpoint}#{dest_filepath}/")
print(f"recon file:{dest_filepath}/{src_filename} output:{dest_resultpath}/")
print(f"transfer file from {dest_endpoint}:{dest_resultpath} to {src_endpoint}#{src_result_path}/")

transfer file from dd916908-0072-11e7-badc-22000b9a448b#/prisma-data1/bicer/workflow/input/extracted_scan300.h5 to 08925f04-569f-11e7-bef8-22000b9a448b#/lus/grand/projects/hp-ptycho/bicer/ptycho/comm_33IDD/globus_automate/transferred/
recon file:/lus/grand/projects/hp-ptycho/bicer/ptycho/comm_33IDD/globus_automate/transferred/extracted_scan300.h5 output:/lus/grand/projects/hp-ptycho/bicer/ptycho/comm_33IDD/globus_automate/result/
transfer file from 08925f04-569f-11e7-bef8-22000b9a448b:/lus/grand/projects/hp-ptycho/bicer/ptycho/comm_33IDD/globus_automate/result to dd916908-0072-11e7-badc-22000b9a448b#/prisma-data1/bicer/workflow/result/


In [8]:
flow_action = flows_client.run_flow(flow_id, flow_scope, flow_input)
print(flow_action)

flow_action_id = flow_action['action_id']

flow_status = flow_action['status']
print(f'Flow action started with id: {flow_action_id}')


while flow_status == 'ACTIVE':
    time.sleep(10)
    flow_action = flows_client.flow_action_status(flow_id, flow_scope, flow_action_id)
    flow_status = flow_action['status']
    # val =  fxs.deserialize(flow_action['exception']) # serialized codes, e.g., exception, can be deserialize using this
    # print(val.e_value)
    print(f'Flow status: {flow_status}')

Starting login with Globus Auth, press ^C to cancel.
GlobusHTTPResponse({'action_id': '00afac42-73f6-4230-88d0-cbda234ceb02', 'completion_time': 'None', 'created_by': 'urn:globus:auth:identity:a478e9c0-d274-11e5-9901-63aaec352b80', 'details': {'code': 'ActionStarted', 'description': 'State Transfer of type Action started', 'details': {'input': {'destination_endpoint_id': '08925f04-569f-11e7-bef8-22000b9a448b', 'source_endpoint_id': 'dd916908-0072-11e7-badc-22000b9a448b', 'transfer_items': [{'destination_path': '/lus/grand/projects/hp-ptycho/bicer/ptycho/comm_33IDD/globus_automate/transferred/extracted_scan300.h5', 'recursive': False, 'source_path': '/prisma-data1/bicer/workflow/input/extracted_scan300.h5'}]}, 'state_name': 'Transfer', 'state_type': 'Action'}, 'time': '2021-02-17T16:01:51.334000+00:00'}, 'start_time': '2021-02-17T16:01:51.295000+00:00', 'status': 'ACTIVE'})
Flow action started with id: 00afac42-73f6-4230-88d0-cbda234ceb02
Flow status: ACTIVE
Flow status: ACTIVE
Flow sta